In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests

from kanon.units import Sexagesimal, Historical, IntegerAndSexagesimal

number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1419, with_units=False)
Hist_table


Months,Entries
IntegerAndSexagesimal,Historical
31 ;,"02 ; 34,37"
59 ;,"04 ; 54,16"
90 ;,"07 ; 28,51"
120 ;,"09 ; 58,31"
151 ;,"12 ; 33,08"
181 ;,"15 ; 02,46"
212 ;,"17 ; 37,23"
243 ;,"20 ; 12,00"
273 ;,"22 ; 41,37"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(12):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)


In [4]:
# moon mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("0;4,59,15,27,7,23,50"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),7)

for i in range(12):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=4):
        Test_recomp[i][1]=daily_mean*Hist_table[i][0]%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=3600*(Hist_table[i][1]-Test_recomp[i][1])
Test_recomp

Months,Entries
IntegerAndSexagesimal,Historical
31 ;,"02 ; 34,37"
59 ;,"04 ; 54,16"
90 ;,"07 ; 28,53"
120 ;,"09 ; 58,31"
151 ;,"12 ; 33,08"
181 ;,"15 ; 02,46"
212 ;,"17 ; 37,23"
243 ;,"20 ; 12,00"
273 ;,"22 ; 41,37"


In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(-600.1666666666666, nan)

In [6]:
B.astype(float).style.background_gradient(axis=None)

,Entries
Months,
31 ;,0.000000
59 ;,0.000000
90 ;,-2.000000
120 ;,0.000000
151 ;,0.000000
181 ;,0.000000
212 ;,0.000000
243 ;,0.000000
273 ;,0.000000


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Jupiter mean months.csv')
B.to_csv('Jupiter mean months.zip', index=False, compression=compression_opts)